# AI - Final - Computer Assignment
## Hossein Entezari Zarch - 810196419
## Summer - 2020


In [47]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error as MSE
from __future__ import unicode_literals
import hazm
import torch as t

 # Introduction
<p dir='rtl'>
    مقدمه: هدف ما در اینجا به طور کلی ساختن یک تابع(مدل پیش‌بینی) است که بتواند با استفاده از داده‌هایی که به عنوان دیتاست به ما داده شده است، به خوبی الگوهای موجود در داده‌ها را کشف کند و آنها را یاد بگیرد. این مدل توانایی خواهد داشت که با دیدن و یادگیری روی داده‌هایی که ما به آن می‌دهیم مثال‌هایی که تا به‌حال ندیده را پیش‌بینی و حل کند.
    <br><br>
    روند کلی: روند کلی ای که ما برای انجام این فرآیند پیش می‌گیریم شامل: ۱-پیش‌پردازش داده‌ها و استخراج ویژگی‌ها ۲-جدا کردن بخشی از داده ها به عنوان داده آزمایش ۳-تعریف مدل و انجام عملیات یادگیری روی آن ۴-پیش‌بینی داده‌های آزمایش و مقایسه با مقادیر واقعی هدف
    <br><br>
</p>

# PreProcess & Feature Extraction
<p dir='rtl'>
    به طور کلی در اینجا هدف ما تبدیل داده‌ها به شکلی است که برای مدل ما قابل فهم‌تر باشند و بیشتر مدل بتواند از آنها اطلاعات یاد بگیرد، به منظور این هدف ما فیلد‌های مختلف داده‌ها را به صورت های مختلف تحلیل کرده و ذخیره می‌کنیم و همچنین در این مرحله داده‌هایی مقادیر نامناسب(داده پرت) هستند را حذف می‌کنیم.
    <br><br>
    encodeing متغیر های گسسته:
    به طور کلی در ساده ترین راه‌ها برای encode
    کردن این نوع داده‌ها، دو روش Labeling, OneHotEncoding
    استفاده م‌شود که به طور خلاصه در روش Labeling 
    هر مقدار از متغیر به یک عدد یکتا مپ می‌شود و آن عدد به جای آن مقدار ذخیره می‌شود و در روش OneHotEncoding
    به ازای هر مقدار خاص از متغیر یک جایگاه اختصاص داده می‌شود که اگر آن مقدار رخ داده جایگاه متناظرش ۱ می‌شود و بقیه جایگاه‌ها صفر می‌مانند.
    <br><br>
    ما در اینجا در encode 
    کردن {برند، شهر} از روش OneHotEncoding 
    استفاده کردیم زیرا با توجه به این که از مدل MultiLayerPerceptronRegressor
    استفاده می‌کنیم و خروجی در این مدل در اصل ترکیبی غیرخطی(وجود تابع غیرخطی بین لایه‌ها) از مقادیر ورودی است لذا مقادیر مختلف معنای فاصله بین‌شان تعریف نمی‌شود و اگر دو مقدار Index
    متناظرشان نزدیک هم باشد یا یکی از دیگری بزرگ‌تر باشد معنایی ندارد ولی این می‌تواند شبکه را به اشتباه بیندازد ولی در روش استفاده شده ما همه مقادیر ارزش و اعتبار یکسان دارند و این تنها وزن‌های شبکه است که میان آنها تمایز ایجاد می‌کند. پش با این استدلالات فیلدهای {برند، شهر} که تعداد مقادیرشان نیز زیاد نیست با OneHotEncoding
    انکود کردیم.
</p>

In [48]:
def one_hot_encode_brand(data):
    enc = OneHotEncoder(handle_unknown='ignore')
    enc_df = pd.DataFrame(enc.fit_transform(data[['brand']]).toarray())
    enc_df.columns = enc.get_feature_names(['brand'])
    data = data.join(enc_df)
    data = data.drop('brand', axis=1)
    return data

def one_hot_encode_city(data):
    enc = OneHotEncoder(handle_unknown='ignore')
    enc_df = pd.DataFrame(enc.fit_transform(data[['city']]).toarray())
    enc_df.columns = enc.get_feature_names(['city'])
    data = data.join(enc_df)
    data = data.drop('city', axis=1)
    return data

# Cycle Encode
<p dir='rtl'>
  در مورد فیلد {روز و ساعت}، به طور کلی ما از اهمیت این داده‌ها در فرآیند یادگیری و میزان تاثیر آنها در یادگیری مدل (میزان وابستگی قیمت به این فیلد) آگاه نیستیم ولی برای اطمینان و با توجه به این که مدل ما در صورت نیاز از این داده استفاده خواهد کرد و یا در غیر این صورت خیر، پس ابتدا باید داده‌ها را به صورتی که برای مدل به راحتی قابل فهم باشد در می‌آوریم  سپس آنها را در داده ذخیره کنیم.
    <br><br>
    Cyclic Encoding:
    می‌دانیم که داده‌های مربوط به تاریخ و ساعت هم توالی دارند و هم تکرار یعنی به عنوان مثال شنبه بسیار به جمعه نزدیک است و این توالی ادامه دارد لذا بهترین راهی که به نظر من رسید، این بود که این داده‌ها را در تابع sin, cos
    که توابعی متناوب هستند مدل کنیم. برای این مورد هر کدام از ساعت و روز را جداگانه با فاصله مساوی زاویه‌ای روی دایره مثلثاتی تعریف کردیم و sin, cos
    متناظر آن زاویه را در نظر گرفتیم.
    <br><br>
    پیاده‌سازی: برای پیاده‌سازی این هدف، ابتدا لیست مقادیر متمایز این متغیر ها را تهیه کردیم و سپس هر کدام از آنها را به ترتیب و با فاصله یکسان روی دایره مثلثاتی قرار داده و لیست سینوس و کیسنوس متناظرشان را ساختیم و در نهایت با مراجه به آن لیست، مقادیر این فیلد را تبدیل و جایگزین فیلد اولیه کردیم.
</p>

In [49]:
def cycle_encode(values, value):
    return [np.sin(2*np.pi*values.index(value)/len(values)), np.cos(2*np.pi*values.index(value)/len(values))]

def make_week_day_cycle_list():
    ans = {}
    values = ['Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    for value in values:
        ans[value] = cycle_encode(values, value)
    return ans

def make_hour_cycle_list():
    ans = {}
    values = ['01AM', '02AM', '03AM', '04AM', '05AM', '06AM', '07AM', '08AM', '09AM', '10AM', '11AM', '12AM',
              '01PM', '02PM', '03PM', '04PM', '05PM', '06PM', '07PM', '08PM', '09PM', '10PM', '11PM', '12PM']
    for value in values:
        ans[value] = cycle_encode(values, value)
    return ans

def created_at_encode(data):
    day_sin = []
    day_cos = []
    sinc = make_week_day_cycle_list()
    
    hour_sin = []
    hour_cos = []
    hours = make_hour_cycle_list()
    
    for element in data['created_at']:
        [day, hour] = element.split()
        day_sin.append(sinc[day][0])
        day_cos.append(sinc[day][1])
        
        hour_sin.append(hours[hour][0])
        hour_cos.append(hours[hour][1])
    new_df = pd.DataFrame({'day_sin': day_sin, 'day_cos': day_cos, 'hour_sin': hour_sin, 'hour_cos': hour_cos})
    data = data.join(new_df)
    data = data.drop('created_at', axis=1)
    return data

# Text Process
<p dir='rtl'>
    به طور مشخص دو فیلد {عنوان، توضیحات} که تنها فیلدهایی هستند که مدل گوشی و وضعیت آن را مشخص می‌کنند قطعا نقش بسازایی در عملکرد شبکه دارند اما این فیلدها به این صورت خام قابل استفاده و ورودی دادن به شبکه نیستند لذا باید با روش مناسب اطلاعات مفیدی از آنها استخراج کنیم.
    <br><br>
    برای استخراج اطلاعات از این دو فیلد، ابتدا تمامی کلمات موجود در هر فیلد را جداگانه می‌خوانیم و تمامی کلمات به همراه تعداد تکرار شان را محاسبه می‌کنیم، سپس به دلیل تعداد زیاد کلمات و همچنین تعداد زیاد کلماتی که تکرار آنها بسیار پایین است، کلمه‌های کم‌تکرار را از لیست کلمات حذف می‌کنیم تا به این وسیله هم تعداد ویژگی ها کم شود و هم با حذف کلمات کم‌تکرار از overfitting
    مدل جلوگیری کنیم (زیرا اگر مثلا کلمه‌ای که یک بار تنها تکرار شده را به عنوان ویژگی به شبکه بدهیم شبکه می‌تواند آن داده را عملا حفظ کند و هر وقت آن ویژگی را دید قیمت آن محصول را خروجی دهد و این کار ارزش یادگیری ندارد).
    <br><br>
    کار با متن: به دلیل این که با متنی مواجه هستیم که بسیار نامتوازن و حاوی فاصله‌های نامناسب است قبل از هر بار خواندن هر متن ابتدا آن را به وسیله hazm.Normalizer()
    نرمالایز می‌کنیم تا کمی از نامتوازنی متن کاسته شود و سپس با استفاده از hazm.tokenize()
    آرایه حاوی کلمات متن را دریافت می‌کنیم و سپس روی هر کلمه hazm.Stemmer()
    اعمال می‌کنیم تا ریشه کلمه بدست آید و این ریشه کلمه معیار ما برای وقوع آن کلمه(ویژگی) در متن خواهد بود.
    <br><br>
    توضیح: علت معیار قرار دادن کلمات به عنوان ویژگی آن است که به طور کلی وقوع بعضی کلمات خاص می‌توانند معنای مثبت یا منفی در متن ایجاد کنند و البته که این روش خطا بسیار دارد ولی با توجه به امکانات ما در تحلیل متن و نوع متن نامتوازنی که در داده‌ها بود تصمیم گرفتم تا صرفا بر اساس وقوع کلمات ویژگی‌هایی را استخراج کنم.
    <br><br>
    در این جا کلماتی که وقوع آنها کمتر بوده از لیست کلی کلمات حذف شدند(برای عنوان، کلمات با تکرار بالای ۱۰۰ و برای توضیحات، کلمات با تکرار بالای ۵۰۰ استفاده شدند) و همچنن کلماتی که تنها یک کاراکتر داشتند نیز به طور کلی حذف شدند همچنین قطعا حذف کلمات پرتکراری که stop word
    هستند و عملا هرجایی می‌توانند بیایند و معنای خاصی به مدل نمی‌دهند می‌توانست بسیار کمک کند تا مدل سبک‌تری ساخته شود اما با توجه به این که کتابخانه این ابزار را نداشت نتوانستم این کار را انجام دهم و البته حضور این کلمات تداخلی در امر یادگیری شبکه ایجاد نمی‌کند زیر شبکه می‌تواند کارکرد آنها یا عدم کارکرد آنها را درک کند.
    <br><br>
    حال پس از نهایی کردن این لیست های کلمات که یکی بر اساس متن title
    و دیگری بر اساس متن desc
    هستند، تقریبا در مجموع به ۴۹۰ کلمه رسیدیم که در این مرحله به هر داده یک بردار oneHot
    به طول تعداد این کلمات اختصاص می‌دهیم تا وقوع این کلمات در هر یک از دو فیلد خود را ذخیره کند و این کار با استفاده از پیمایش روی داده ها انجام می‌شود.
    <br><br>
    به طور کلی علت این که متن های موجود در دو فیلد را جداگانه تحلیل و بررسی کردم این بود که به نظرم آمد دامنه کلمات این دو فیلد می‌تواند متفاوت باشد و وقوع کلمه در هر یک ممکن است بتواند معنای متفاوتی ایجاد کند.
</p>

In [50]:
def generate_library(data):
    normalizer = hazm.Normalizer()
    stemmer = hazm.Stemmer()
    
    title_wl = {}
    for sentence in data['title']:
        normal_sentence = normalizer.normalize(sentence)
        words = hazm.word_tokenize(normal_sentence)
        for word in words:
            sw = stemmer.stem(word)
            if sw not in title_wl:
                title_wl[sw] = 0
            title_wl[sw] += 1
    title_wl = {key:val for key, val in title_wl.items() if val > 100 and len(key) > 1}
    title_wl = {k: v for k, v in sorted(title_wl.items(), key=lambda item: item[1])}
    
    desc_wl = {}
    for sentence in data['desc']:
        normal_sentence = normalizer.normalize(sentence)
        words = hazm.word_tokenize(normal_sentence)
        for word in words:
            sw = stemmer.stem(word)
            if sw not in desc_wl:
                desc_wl[sw] = 0
            desc_wl[sw] += 1
    desc_wl = {key:val for key, val in desc_wl.items() if val > 500 and len(key) > 1}
    desc_wl = {k: v for k, v in sorted(desc_wl.items(), key=lambda item: item[1])}
                
    return title_wl.keys(), desc_wl.keys()

def encode_text(data):
    normalizer = hazm.Normalizer()
    stemmer = hazm.Stemmer()
    
    title_words, desc_words = generate_library(data)
    title_data = {}
    desc_data = {}
    for word in title_words:
        title_data['title_'+word] = [0 for _ in data['title']]
    for (i,sentence) in enumerate(data['title']):
        normal_sentence = normalizer.normalize(sentence)
        words = hazm.word_tokenize(normal_sentence)
        for word in words:
            sw = stemmer.stem(word)
            if 'title_'+sw in title_data:
                title_data['title_'+sw][i] = 1
    
    for word in desc_words:
        desc_data['desc_'+word] = [0 for _ in data['desc']]
    for (i, sentence) in enumerate(data['desc']):
        normal_sentence = normalizer.normalize(sentence)
        words = hazm.word_tokenize(normal_sentence)
        for word in words:
            sw = stemmer.stem(word)
            if 'desc_'+sw in desc_data:
                desc_data['desc_'+sw][i] = 1

    
    title_data = pd.DataFrame(title_data)
    desc_data = pd.DataFrame(desc_data)
    data = data.join(title_data)
    data = data.join(desc_data)
    data = data.drop(['title', 'desc'] , axis=1).dropna()
    return data

# Data Normalization
<p dir='rtl'>
    به طور کلی برای آموزش هر مدلی که بر پایه شبکه عصبی است باید داده‌ها را نرمالایز کنیم، که در این فرایند داده‌های هر فیلد به مقادیر بین {-۱و ۱} اسکیل می‌شوند به طوری که میانگین شان ۰ و واریانس شان ۱ شود و این عمل باعث می‌شود مدل روی این داده‌ها خوش فرم‌تر باشد و لذا بسیار زودتر و بهینه‌تر مدل همگرا شود، همچنین به  تحربه دیده شده که در بسیاری از مدل‌های شبکه عصبی که داده‌های یادگیری شان نرمال نشده شبکه هیچ چیزی یاد نگرفته و هیچ بهبودی در عملکردش نداشته است.
    <br><br>
    ما این عملیات نرمال‌سازی را پس از اتمام تمامی مراحل پیش‌پردازش روی داده‌ها، روی تمامی داده‌ها و تمامی فیلد‌های آنها به جز فیلد قیمت که فیلد هدف بود و ورودی مدل نبود انجام دادیم تا از این طریق اطمینان یابیم که روی تمامی داده‌هایمان دقیقا یک عملیات اسکیل انجام گرفته است. همچنین این تابع که نرمالایز کردن داده‌ها را انجام می‌دهد مدل scaler
    را نیز خروجی می‌دهد تا بتوانیم برای ورودی دادن داده‌هایی که قبلا نرمال نشده‌اند به شبکه ترین شده، ابتدا آنها را با گذراندن از این مدل نرمال کنیم.
    
</p>

In [51]:
def normalize_data(data):
    columns = data.columns
    scaler = StandardScaler()
    scaler.fit(data)
    x = scaler.transform(data)
    x = pd.DataFrame(x, columns=columns)
    return x, scaler

# Read Data & Perfrom PreProcess
<p dir='rtl'>
    در این مرحله داده‌ها را از فایل می‌خوانیم و توابعی که بالاتر تعریف کرده بودیم را روی آنها صدا می‌زنیم تا عملیات آماده‌سازی داده‌ها انجام گیرد. همچنین در این مرحله داده‌هایی که قیمت شان زیر ۱۰۰۰۰ را حذف می‌کنیم زیرا قیمت به این پایینی منطقی نیست و احتمالا یا داده قیمت ندارد و مقدارش -۱ است یا قیمت اشتباه وارد شده است.
</p>

In [52]:
data = pd.read_csv('mobile_phone_dataset.csv').drop(['Unnamed: 0'], axis=1).dropna().reset_index()
data = one_hot_encode_brand(data)
data = one_hot_encode_city(data)
data = created_at_encode(data)
data = data[data['price'] > 10000].reset_index()
data = encode_text(data)

# Perform Data Normalization
<p dir='rtl'>
در این قسمت ستون قیمت از داده‌را به عنوان هدف(y
)
تعیین می‌کنیم وبرای ویژگی‌های داده‌ها نیز ستون قیمت را حذف کرده و بقیه را به عنوان
X
تعیین می‌کنیم.
    <br><br>
    فیلد قیمت‌هارا نرمالایز نکردیم زیرا نیازی به این کار نبود و این فیلد جزو ورودی‌های مدل و ویژگی‌های داده‌ها نیست و عملا مدل روی این متغیر تعریف نمی‌شود.
</p>

In [40]:
# y = pd.DataFrame(data['price'])
y = data['price']
# data = data.drop('price', axis=1)
X, data_scaler = normalize_data(data)

# Train/Test Set
<p dir='rtl'>
مطابق همیشه باید داده‌ها به دو بخش یادگیری و آزمایش تقسیم شوند تا بخشی از داده ها موجود باشند که مدل طی فرایند یادگیری آن‌را نمی‌بیند و از این داده‌های جدید برای مدل به منظور امتحان کردن و تست کارکرد مدل و محاسبه خطا استفاده می‌شود.
    <br><br>
    
</p>

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [42]:
model = MLPRegressor(hidden_layer_sizes=(1000, 200, 10))
model.fit(X_train, y_train)

/home/ho/ai/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:587: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPRegressor(hidden_layer_sizes=(1000, 200, 10))

In [43]:
y_pred = model.predict(X_test)
loss = MSE(y_test, y_pred)
print(loss)

1114402786.2214222


In [45]:

for (i, j) in enumerate(y_test):
    print(y_pred[i], j)

317900.05437801004 280000
852186.747363078 850000
448940.5253708446 450000
282428.44987252814 280000
654967.6048426987 650000
412042.99005060387 400000
387698.8872489152 400000
1999016.7897801988 2000000
169194.01799548938 130000
1230361.2070607194 1200000
607836.5758280515 480000
447633.51494193345 450000
554208.0070594971 500000
527235.0407991267 550000
1794642.8743095875 1800000
1850096.846670305 1850000
291999.39372577856 250000
1352125.0493491413 1350000
708175.6462202971 720000
107302.99978149637 90000
270164.8707264298 260000
144078.72634885687 140000
91656.33633866628 100000
726487.7034970241 750000
75595.59070839205 50000
1791693.3912469707 1800000
761953.2996443313 800000
294885.64980512497 300000
122650.70765729333 130000
328115.48521093116 250000
427372.468496528 400000
424718.7349485042 400000
995192.0088971144 1000000
1524339.0549070975 1500000
1483313.780344328 1550000
508047.3523178099 500000
370861.60166190704 310000
425283.8093077846 400000
140573.36382421467 130000
4

502514.3861499468 500000
100100.76114841945 60000
504640.120634411 500000
227793.12326367485 200000
277375.35444369126 250000
189030.26265596587 170000
240127.99927814526 250000
892735.1599982878 900000
309571.38382784656 280000
737867.2901648594 700000
105872.44342767117 100000
288982.3506199555 350000
887485.1391978828 890000
857808.6370832947 870000
639093.8757848941 650000
699148.4048968303 700000
944871.1641062811 950000
1665440.005087222 1670000
521254.05154394725 550000
591471.6972439323 900000
619773.5501957277 600000
362058.93033924216 350000
715693.1570558717 700000
634000.7659242131 600000
158620.59459234058 150000
48758.63549493552 45000
915064.0768802406 930000
120494.94302494283 100000
602160.8914462909 600000
1056334.1303107473 1050000
406357.2809906741 400000
182525.4686296386 190000
315468.77165782225 265000
187432.1292194049 200000
677939.303986581 680000
49040.6126805496 45000
115710.23661877734 100000
626132.7353417239 620000
127956.97985125701 100000
965606.1043666

878273.0497732495 900000
131280.39572908002 110000
259323.7717814735 230000
269070.7254325613 160000
525597.2753946629 535000
658055.9818205569 650000
265219.4538856804 250000
690851.0798328203 740000
122372.8118503304 100000
797622.0879393241 810000
562775.5080900311 565000
374856.44827318646 380000
316802.42963886115 320000
881716.580721858 880000
252740.58559926334 250000
1498246.052096332 1500000
541452.0416168394 590000
1730299.697510598 1700000
629910.8081703332 630000
177534.47893807085 170000
1052620.0197764358 1050000
477350.6590078983 480000
454172.21651116986 400000
1550158.151703097 1550000
719676.9194294801 650000
48985.89529685078 40000
178973.4952029147 110000
260704.1772688312 270000
1855267.4657301474 1850000
390518.10959523753 319000
60658.071021274984 35000
366043.4513082862 350000
1027173.9052585333 1020000
1597706.6804062175 1600000
125397.88021830838 130000
689092.4905832825 700000
467182.2590013702 790000
223861.16892572743 150000
428077.46972725226 400000
569645

395870.99776425154 430000
399505.95182843186 370000
996714.1968223184 1000000
535476.99441886 550000
1491174.3958128807 1500000
387731.0117997161 400000
282022.8081731037 200000
1583882.8203360597 1600000
1325727.934294339 1330000
287712.8384717004 40000
356795.56738905143 350000
513829.10384737153 500000
77873.12209766598 45000
687697.8789305979 680000
425609.5262219853 410000
704025.7892713293 700000
339566.4775129062 300000
999860.0659890212 999000
391458.82464143616 350000
189802.07513099196 250000
1352983.7502377355 1350000
632707.8211761583 700000
465840.13146993966 480000
1063942.5985116425 1050000
1560252.5737240417 1550000
383329.5105604399 400000
299158.81695922755 350000
74364.39877524976 68000
339355.1769491418 280000
1200628.0620986423 1200000
496555.298647057 500000
417217.7153432346 410000
427770.1141458339 345000
1356516.4296413658 1350000
1199132.291944086 1200000
348994.2627331811 360000
599269.9510032869 580000
833716.8826300177 800000
990728.9350466237 1000000
79422

1722501.5863001149 1710000
395763.67674846895 400000
704054.7908352187 700000
245655.09501374068 240000
521805.8041763226 510000
2104825.17808602 2100000
389842.4634836585 400000
123727.62283292886 100000
496185.0088071197 500000
130843.24742998646 120000
490985.8875713676 490000
157044.62205757885 150000
139933.01099163698 110000
1709414.0992230882 1700000
235400.7742636981 200000
592652.0747978962 620000
183697.35391466666 210000
481436.16301782045 500000
159692.72128405335 150000
131287.90022914106 90000
253537.3360536115 290000
669815.5163880654 700000
511534.6821823386 500000
529393.875395928 550000
364640.8214697081 370000
655544.7894709154 680000
186404.8111024239 200000
723024.9816920997 760000
993851.9992849946 1000000
96993.62934037765 50000
379413.7547300622 340000
806697.588428041 830000
803302.6596073604 800000
577350.1841160752 550000
258812.8487068496 240000
572097.1496393109 550000
204050.4362298251 190000
476725.28077154985 480000
1586018.315103269 1600000
1874094.9709

130626.10637768397 140000
2142881.972390752 2150000
222172.17191828944 240000
157661.76518400246 190000
1491243.0070752325 1500000
345252.40970853803 320000
1996680.6744074712 2000000
401983.47201606695 400000
142148.54646426486 150000
961802.972678758 950000
2149659.567332245 2150000
684552.8741411384 700000
519496.0247244187 550000
411282.1127604474 400000
1682160.8857507263 1700000
381904.02206932526 370000
713461.6538569457 700000
609469.1387465674 640000
77818.52747988877 90000
250558.77026188787 250000
310602.1834365779 320000
1494707.3830000854 1480000
719994.5369618074 650000
808039.3961279744 820000
212237.59303336625 160000
1200884.2771172335 1190000
124733.41102836056 100000
1638288.2589201848 1650000
185293.7081371551 150000
494189.5480626481 500000
533608.1414636084 500000
277477.55800291576 250000
1800243.304421952 1800000
891604.1528526142 890000
195860.00289380673 220000
128606.02268475671 100000
255392.82914649858 235000
275484.3974473553 250000
96023.6861279385 100000

In [ ]:
      30, 10: 303653716240.4726
300, 100, 10: 300258590157.21277
           1: 778024447261.5586
            
           1: 770299151209.7996
      30, 10: 303144475151.69855
            :    569367657.8633283
1000,200,10 :    424475009.4569041
              416056424878.6864
               96284887133.45554
               89591426019.40828
              104802452655.02263
              101659684232.63559
              208122353123.70355
                1114402786.2214222